# Ejercicio EvCont4 (b) - Compresión de texto con LZZ7

* Óscar Sementé Solà
* Abdelkarim Azzouguagh Ouniri
* Rodrigo Cabezas Quirós

In [33]:
import math, re, time

class LZZ7Compressor:
    
    def __init__(self, win, wsld):
        self.win = win
        self.wsld = wsld
        self.win_b = int(math.log2(win))
        self.wsld_b = int(math.log2(wsld))
    
    def __is_power_of_two(self, v):
        return math.log2(v).is_integer()
    
    def __ld_to_bin(self, l, d, useLD):
        """
        Transforma (L, D) a formato binario de longitud fija.
        """
        b_win = lambda x : ''.join(reversed( [str((x >> i) & 1) for i in range(self.win_b)]))
        b_wsld = lambda x : ''.join(reversed( [str((x >> i) & 1) for i in range(self.wsld_b)]))
        if useLD:
            return (l, d)
        return b_win(l) + b_wsld(d)
    
    def __match_pattern(self, slide, inp, useLD):
        """
        Busca un patrón de la ventana de entrada en la ventana deslizante. Si lo encuentra devuelve
        (L, D) en decimal o en formato binario (según valor de 'useLD') y la longitud del patrón
        encontrado. Si no lo encuentra devuelve False y 1 (incremento del offset).
        """
        occ, seqs = [], [inp[:i] for i in range(len(inp), 0, -1)]
        while seqs:
            current = seqs.pop(0)
            occ = [(x.start(0), x.end(0)) for x in re.finditer(current, slide)]
            if occ:
                si, ei = occ[-1]
                return self.__ld_to_bin(len(slide[si:ei]), len(slide) - si, useLD), ei - si
        return False, 1
    
    def compress_bin(self, m, useLD=False):
        """
        Compresión de una string binaria a partir de la configuración del compresor.
        """
        if not self.win <= self.wsld and not len(m) >= self.win + self.wsld:
            """
            Que los bits de conf. sean potencia de 2 no tiene ningún sentido. El ejercicio evaluado en
            clase tenía por parametros 8 y 6 bits. Aplicando la norma de la potencia de 2, con 6 bits no
            funcionaria el algoritmo. Así que hemos aplicado esta política.
            or \ not self.__is_power_of_two(self.win) \ or not self.__is_power_of_two(self.wsld):
            """
            return -1
        # Se aplica método de inserción de bit, para los casos problematicos.
        offset, m = 0, self.__add_insertion_bits(m)
        ret = [m[:self.wsld]]
        while offset + self.wsld + self.win <= len(m):
            # Búsqueda de patrón de la ventana de entrada en la deslizante.
            slide = m[offset:offset + self.wsld]
            inp = m[offset + self.wsld:offset + self.wsld + self.win]
            found, flen = self.__match_pattern(slide, inp, useLD)
            if found:
                # Si se encuentra se añade al retorno.
                ret.append(found)
            else:
                # Si no se guarda el símbolo (en formato (L, D) tal y como especifica el enunciado).
                ret.append(self.__ld_to_bin(1, 1, useLD))
            offset -= -flen
        # Si al acabar quedan bits fuera de las ventanas, se añaden al final de la cadena.
        if offset < len(m):
            ret.append(m[offset + self.wsld:])
        if useLD:
            return ret
        return "".join(ret)
    
    def uncompress_bin(self, m):
        """
        Descompresión de una string binaria a partir de la configuración del compresor.
        """
        ret = m[:self.wsld]
        div = [m[i:i+self.wsld_b+self.win_b] for i in range(self.wsld, len(m), self.wsld_b + self.win_b)]
        lf = lambda x: 2**self.win_b if x == 0 else x
        df = lambda x: 2**self.wsld_b if x == 0 else x
        while div:
            c = div.pop(0)
            if len(c) < self.wsld_b + self.win_b:
                ret = ret + c
            else:
                l, d = lf(int(c[:self.win_b], 2)), df(int(c[self.win_b:], 2))
                ret = ret + ret[len(ret) - d: len(ret) + l - d]
        return self.__remove_insertion_bits(ret)
    
    def __add_insertion_bits(self, m):
        """
        Aplica método de inserción de bit para las entradas con un solo carácter.
        """
        asd = [m[i:i + self.wsld - 1] for i in range(0, len(m), self.wsld - 1)]
        opposite = lambda x: "0" if x == "1" else "1"
        for i in range(len(asd)):
            if asd[i] == asd[i][0] * len(asd[i]) and len(asd[i]) == self.wsld - 1:
                asd[i] = asd[i] + opposite(asd[i][0])
        return "".join(asd)
    
    def __remove_insertion_bits(self, m):
        """
        Desaplica método de inserción de bit para las entradas con un solo carácter.
        """
        opposite = lambda x: "0" if x == "1" else "1"
        for i in range(0, len(m), self.wsld):
            p = m[i:i + self.wsld]
            if p[:-1] == p[0] * len(p[:-1]) and p[-1] == opposite(p[0]):
                m = m[:i + self.wsld-1] + m[i+self.wsld:]
        return m
    
    def __text_to_bin(self, m):
        return "".join([bin(i)[2:].zfill(8) for i in bytearray(m, "utf-8")])
    
    def __bin_to_text(self, b):
        return "".join([chr(int(b[i:i+8], 2)) for i in range(0, len(b), 8)])
    
    def compress_text(self, m):
        startTime = time.time()
        b = self.__text_to_bin(m)
        c = self.compress_bin(b)
        print("[Info] Compression took {} seconds".format(time.time() - startTime))
        return c
    
    def uncompress_text(self, m):
        startTime = time.time()
        uBin = self.uncompress_bin(m)
        u = self.__bin_to_text(uBin)
        print("[Info] Uncompression took {} seconds".format(time.time() - startTime))
        return u
    
    def compress_text_from_file(self, fn):
        with open(fn, "r") as f:
            text = f.read()
        f.close()
        return self.compress_text(text)

#### [Apartado 1] Modificar el compresor para que sea capaz de leer ficheros de texto y devolverlos como una string binaria procesable por vustro compresor LZZ7. Ha de calcular el tiempo de compresión y descompresión. Comprobad el correcto funcionamiento del programa.

In [35]:
comp = LZZ7Compressor(4, 8)
msg = "It is practically impossible to teach good programming to students that have had a prior exposure to BASIC: as potential programmers they are mentally mutilated beyond hope of regeneration. - Edsger Dijkstra"
asd = comp.compress_text(msg)
print("\nCompressed:\t{}\n\nUncompressed:\t{}".format(asd, comp.uncompress_text(asd)))

[Info] Compression took 0.0070018768310546875 seconds
[Info] Uncompression took 0.0010004043579101562 seconds

Compressed:	0100100110010010011110001010010011001000101001111000010010110111000000111001001001110010111110100101001000000000011101000001010011110000001111100000111000101001001110000011111101111100010100000100100000011011010110100100000000001110100111110111101110010111111000010010110110000011011111011010001001100101100011011001011110011101000000000011011100110101011011110000011100000110110000011100100100010110100110110111101001010011010000000111111010010101101101001110010111110100101001101000000011101100111001000000100100000000111000001001101010010011001000110000000111111000101001010111011111110000000000000000101110100100110010100110101110100101001000000000011101000001101100101100111110000000101001010111011111001111000000110110000000000000001110100000110111110000000111000010110011100101111101001010011010000000111111010010101101101001110010111110100101001001010000010100010100100

## -> Se deben usar los ficheros 'txt' adjuntos a este notebook (se han eliminado un par de espacios innecesarios y problematicos al codificar).

#### [Apartado 2] Comprimir fichero "hamlet_short.txt"

In [36]:
fname = "hamlet_short.txt"
asd = comp.compress_text_from_file(fname)
print("\n", comp.uncompress_text(asd))

[Info] Compression took 0.04301881790161133 seconds
[Info] Uncompression took 0.009002447128295898 seconds

 BERNARDO Who's there?
FRANCISCO Nay, answer me: stand, and unfold yourself.
BERNARDO Long live the king!
FRANCISCO Bernardo?
BERNARDO He.
FRANCISCO You come most carefully upon your hour.
BERNARDO 'Tis now struck twelve; get thee to bed, Francisco.
FRANCISCO For this relief much thanks: 'tis bitter cold, And I am sick at heart.
BERNARDO Have you had quiet guard?
FRANCISCO Not a mouse stirring.
BERNARDO Well, good night. If you do meet Horatio and Marcellus, The rivals of my watch, bid them make haste.
FRANCISCO I think I hear them. Stand, ho! Who's there?

Enter HORATIO and MARCELLUS

HORATIO Friends to this ground.
MARCELLUS And liegemen to the Dane.
FRANCISCO Give you good night.
MARCELLUS O, farewell, honest soldier: Who hath relieved you?
FRANCISCO Bernardo has my place. Give you good night.

Exit

MARCELLUS Holla! Bernardo!
BERNARDO Say, What, is Horatio there?
HORATIO A pi

#### [Apartado 3] Comprimir fichero "quijote_short.txt"

In [37]:
fname = "quijote_short.txt"
asd = comp.compress_text_from_file(fname)
print("\n", comp.uncompress_text(asd))

[Info] Compression took 0.03600001335144043 seconds
[Info] Uncompression took 0.009001731872558594 seconds

 En un lugar de la Mancha, de cuyo nombre no quiero acordarme, no ha mucho tiempo que vivÃ­a un hidalgo de los de lanza en astillero, adarga antigua, rocÃ­n flaco y galgo corredor.
Una olla de algo mÃ¡s vaca que carnero, salpicÃ³n las mÃ¡s noches, duelos y quebrantos los sÃ¡bados, lentejas los viernes, algÃºn palomino de aÃ±adidura los domingos, consumÃ­an las tres partes de su hacienda.
El resto della concluÃ­an sayo de velarte, calzas de velludo para las fiestas con sus pantuflos de lo mismo, los dÃ­as de entre semana se honraba con su vellori de lo mÃ¡s fino.
TenÃ­a en su casa una ama que pasaba de los cuarenta, y una sobrina que no llegaba a los veinte, y un mozo de campo y plaza, que asÃ­ ensillaba el rocÃ­n como tomaba la podadera.
Frisaba la edad de nuestro hidalgo con los cincuenta aÃ±os, era de complexiÃ³n recia, seco de carnes, enjuto de rostro; gran madrugador y amigo 